In [2]:
use std::fs::File;
use std::io::BufReader;
use std::io::BufRead;

let f = File::open("../train/day02-2.txt").unwrap();
let reader = BufReader::new(f);
let lines: Vec<String> = reader.lines().map(|l| l.unwrap()).collect();
lines

["Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green", "Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue", "Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red", "Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red", "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"]

In [3]:
#[derive(Debug)]
struct Bag {
    blue: i32,
    red: i32,
    green: i32,
}

impl Bag {
    fn new(blue: i32, red: i32, green: i32) -> Bag {
        Bag { blue, red, green }
    }

    fn from_string(s: &str) -> Result<Bag, &'static str> {
        let mut blue = 0;
        let mut red = 0;
        let mut green = 0;

        for part in s.split(',') {
            let parts: Vec<&str> = part.trim().split_whitespace().collect();
            if parts.len() != 2 {
                return Err("Invalid input format");
            }

            let count = parts[0].parse::<i32>().map_err(|_| "Invalid count")?;
            match parts[1] {
                "blue" => blue = count,
                "red" => red = count,
                "green" => green = count,
                _ => return Err("Unknown color"),
            }
        }

        Ok(Bag::new(blue, red, green))
    }

    fn power(&self) -> i32 {
        self.blue * self.green * self.red
    }
}

let example = Bag::from_string("8 green, 6 blue, 20 red").unwrap();
example

Bag { blue: 6, red: 20, green: 8 }

In [4]:
#[derive(Debug)]
struct Game {
    id: i32,
    bags: Vec<Bag>,
}

impl Game {
    fn from_string(s: &str) -> Result<Game, &'static str> {
        let parts: Vec<&str> = s.split(':').collect();
        if parts.len() != 2 {
            return Err("Invalid input format");
        }

        let id = parts[0].split_whitespace().last().ok_or("Invalid ID")?
            .parse::<i32>().map_err(|_| "ID is not a number")?;

        let bag_parts = parts[1].split(';');
        let mut bags = Vec::new();

        for part in bag_parts {
            match Bag::from_string(part) {
                Ok(bag) => bags.push(bag),
                Err(e) => return Err(e),
            }
        }

        Ok(Game { id, bags })
    }

    fn max_values(&self) -> Bag {
        let mut max_blue = 0;
        let mut max_red = 0;
        let mut max_green = 0;

        for bag in &self.bags {
            max_blue = max_blue.max(bag.blue);
            max_red = max_red.max(bag.red);
            max_green = max_green.max(bag.green);
        }

        Bag::new(max_blue, max_red, max_green)
    }
}

let game = Game::from_string("Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green\n").unwrap();
game

Game { id: 1, bags: [Bag { blue: 3, red: 4, green: 0 }, Bag { blue: 6, red: 1, green: 2 }, Bag { blue: 0, red: 0, green: 2 }] }

In [5]:
game.max_values()

Bag { blue: 6, red: 4, green: 2 }

In [6]:
game.max_values().power()

48

In [7]:
let mut games = Vec::new();
for line in &lines {
    let game = Game::from_string(&line).unwrap();
    games.push(game);
};
games

[Game { id: 1, bags: [Bag { blue: 3, red: 4, green: 0 }, Bag { blue: 6, red: 1, green: 2 }, Bag { blue: 0, red: 0, green: 2 }] }, Game { id: 2, bags: [Bag { blue: 1, red: 0, green: 2 }, Bag { blue: 4, red: 1, green: 3 }, Bag { blue: 1, red: 0, green: 1 }] }, Game { id: 3, bags: [Bag { blue: 6, red: 20, green: 8 }, Bag { blue: 5, red: 4, green: 13 }, Bag { blue: 0, red: 1, green: 5 }] }, Game { id: 4, bags: [Bag { blue: 6, red: 3, green: 1 }, Bag { blue: 0, red: 6, green: 3 }, Bag { blue: 15, red: 14, green: 3 }] }, Game { id: 5, bags: [Bag { blue: 1, red: 6, green: 3 }, Bag { blue: 2, red: 1, green: 2 }] }]

In [8]:
for game in &games {
    println!("ID {}. Power: {}", &game.id, game.max_values().power());
}

ID 1. Power: 48
ID 2. Power: 12
ID 3. Power: 1560
ID 4. Power: 630
ID 5. Power: 36


()

In [9]:
let mut power_sum = 0;
for game in &games {
    power_sum += game.max_values().power();
}
power_sum

2286